In [7]:

from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

### Preprocess data into Vector DataBase

In [8]:
path="嘸蝦米輸入法.pdf"
loader = PyMuPDFLoader(path)
pdf_data = loader.load()

Text splitter: `RecursiveCharacterTextSplitter`

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=5)
all_splits = text_splitter.split_documents(pdf_data)

The mebedding model to use. Follow the instruction to use `all-MiniLM-L6-c2`

In [10]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs)

In [13]:
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=all_splits, embedding=embedding, persist_directory=persist_directory)

### Model
There are serveral ways to hold the model so that we can execute it:

#### Llamafactory
Use the command (`API_PORT=8000 llamafactory-cli api model_info.yaml`) to run the server:


In [27]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(openai_api_key='None', openai_api_base='http://0.0.0.0:8000/v1')

In [29]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Japanese. Translate the user sentence.",
    ),
    ("human", "Nice to meet you! What a wonderful day!"),
]
response = llm.invoke(messages)
print(response)

content="Here's the translation:\n\n「はじめまして！今日はとても素敵な日ですね！」\n\nBreakdown:\n\n* 「はじめまして！」 (Nice to meet you!)\n* 「今日は」 (Today is)\n* 「とても」 (very)\n* 「素敵な」 (wonderful)\n* 「日ですね」 (day, polite suffix)" response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 41, 'total_tokens': 120}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-2a300d08-ebb3-45b8-a1c9-0639029c1ffd-0'


#### Llamacpp

In [30]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = LlamaCpp(
    model_path="Llama3.1-8B-Instruct-hf",
    n_gpu_layers=100,
    n_batch=512,
    n_ctx=2048,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
)

/tmp/ipykernel_1192598/1736967810.py:4: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = LlamaCpp(
gguf_init_from_file: invalid magic characters '   '
llama_model_load: error loading model: llama_model_loader: failed to load model from Llama3.1-8B-Instruct-hf

llama_load_model_from_file: failed to load model


ValidationError: 1 validation error for LlamaCpp
__root__
  Could not load Llama model from path: Llama3.1-8B-Instruct-hf. Received error Failed to load model from file: Llama3.1-8B-Instruct-hf (type=value_error)